In [5]:
import numpy as np
import pandas as pd

from sklearn.metrics import mean_absolute_error
from scipy.stats import uniform, randint
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.model_selection import RandomizedSearchCV,train_test_split
from sklearn import preprocessing
import xgboost as xgb

#### Modelo: XGB

In [6]:
def mean_target_encoding(train, nombrefeat, nombretarget):
        
    temp = train.groupby(nombrefeat)[nombretarget].transform(np.mean) #Saco promedio
    temp = preprocessing.scale(temp) #Normalizo
    train[nombrefeat + "_mean"] = temp #Dejo en el DF
    
    return train

In [7]:
def mean_target_decoding(x_test, nombrefeat, x_train):
    
    nombrefeatmean = nombrefeat + "_mean"
    
    temp = x_train.loc[:,[nombrefeat,nombrefeatmean]]
    temp = temp.set_index(nombrefeat)
    temp = temp.drop_duplicates()
    temp = temp.T.squeeze()
    values = x_test[nombrefeat].map(temp)
    x_test[nombrefeatmean] = values 

    return x_test

In [8]:
train = pd.read_csv("train.csv")
x_test = pd.read_csv("test.csv")

In [9]:
########### FEATURES DE TRAIN ################


train = train.drop(['descripcion', 'titulo', 'direccion', 'fecha', 'id', 'antiguedad',
                    'lng', 'tipodepropiedad'], axis = 1)

#Adiciono servicios
train['servicios'] = train['piscina'] + train['gimnasio'] + train['usosmultiples']

#Elimino los residuos
train = train.drop(['piscina', 'gimnasio','usosmultiples','escuelascercanas', 
                    'centroscomercialescercanos'], axis = 1)

y_train = train['precio']
temporal = train

#Calculo los mean target
x_train = mean_target_encoding(train, "provincia", "precio")
x_train['precio'] = y_train
x_train = mean_target_encoding(x_train, "ciudad", "precio")

########### FEATURES DE TEST ################

#Adiciono servicios
x_test['servicios'] = x_test['piscina'] + x_test['gimnasio'] + x_test['usosmultiples']

#Se los asigno a los test (NO LOS CALCULO CON ELLOS!!!!!!)
x_test = mean_target_decoding(x_test, "provincia", temporal)
x_test = mean_target_decoding(x_test, "ciudad", temporal)

#Elimino los residuos
x_test = x_test.drop(['piscina', 'gimnasio','usosmultiples','escuelascercanas', 'fecha',
                  'centroscomercialescercanos', 'titulo', 'direccion', 'descripcion',
                  'id', 'antiguedad', 'lng','tipodepropiedad'], axis = 1)

########### LIMPIO LOS QUE HICE MEAN ################

x_train.drop(["precio","provincia", "ciudad"], axis=1, inplace = True)
x_test.drop(["provincia", "ciudad"], axis=1, inplace = True)



In [10]:
import xgboost as xgb

xgb_model = xgb.XGBRegressor(base_score=0.5, 
                             booster='gbtree', 
                             colsample_bylevel=1,
                             colsample_bynode=1, 
                             colsample_bytree=0.7, 
                             gamma=0.2,
                             importance_type='gain', 
                             learning_rate=0.1, 
                             max_delta_step=0,
                             max_depth=8, 
                             min_child_weight=3, 
                             missing=None, 
                             n_estimators=250,
                             n_jobs=1, 
                             nthread=-1, 
                             objective='reg:linear', 
                             random_state=0,
                             reg_alpha=0, 
                             reg_lambda=1, 
                             scale_pos_weight=1, 
                             seed=None,
                             silent=None, 
                             subsample=0.7, 
                             verbosity=1)

In [11]:
import timeit
start_time = timeit.default_timer()


#Fitteo
xgb_model.fit(x_train, y_train)


elapsed = timeit.default_timer() - start_time

/home/ailen-magali/.local/lib/python3.5/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/ailen-magali/.local/lib/python3.5/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[16:05:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [12]:
elapsed

118.50774515500234

In [49]:
#Predigo
prediccion = xgb_model.predict(x_test)

In [50]:
test_original = pd.read_csv("test.csv")
predicciones = test_original.loc[:, ['id','ciudad']]
predicciones.rename(columns = {'ciudad' : 'target'}, inplace = True)
predicciones['target'] = prediccion
predicciones = predicciones.set_index('id')

In [51]:
predicciones.to_csv(r'prediccionXGB7.csv')